# Reinventing the wheel

I wanted to try to derive the ideas of machine learning for myself. I do have some knowledge (so it's not completely independent of course), but I want to try to derive a lot of the ideas myself.

The first sample problem is to write a machine learning algorithm to learn the xor function.

So here's the xor logic table:

| x0 | x1 | y |
|----|----|---|
| 0  | 0  | 0 |
| 0  | 1  | 1 |
| 1  | 0  | 1 |
| 1  | 1  | 0 |

So we have a 2 input function that returns 1 value - something like this:

<img src="images/xor_network1.png"  style="width: 50%; height: 50%" />

My initial thought is that we can convolute through some linear model, and adjust coefficients to get the right answer.

So the equation would look something like this:

$W0*x0 + W1*x1 = y$

You could put this into matrix form and add a bias coefficient, in which case it would look like this:

$Ax +b = y$ where A is the vector containing the 2 weights, x is a free variable to hold the features - x0 and x1, and y is the vector of labels) - so something like this:

<img src="images/xor_matrix_form1.png"  style="width: 50%; height: 50%" />

Before trying to write an algorithm to optimize the values of W0 and W1, let's first make sure this model is powerful enough to work for the xor function. After thinking for a minute, I was able to determine that the values W0 = 1 and W2 = -1 would be good values, since if we plug those weights into the equation, $W0*x0 + W1*x1 = y$, we get $1*x0 + -1*x1$, and that should come out to the values of y (or at least, the value should be > 0.5 for (0, 1) and (1, 0), and < 0.5 for (0, 0) and (1, 1) ). And it turns out, that works:
* for x0=0 and x1=1: `1*0 + -1*0 = 0`
* for x0=0 and x1=1: `1*0 + -1*1 = -1`
* for x0=0 and x1=1: `1*1 + -1*0 = 1`
* for x0=0 and x1=1: `1*0 + -1*0 = 0`

Shoot - when I first ran the numbers in my head, I was thinking the 2nd equation came out to 1, not -1. And actually, graphing it out, xor would look like this:

<img src="images/xor_graph.png"  style="width: 400px; height: 400px;" />

And as you can clearly see, that is not linearly separable. That is why, when I tried a linear classifier, it didn't work:

In [20]:
# Note that I'm making 10x repetitions of the logic table.
X = [[0, 0],
     [0, 1],
     [1, 0],
     [1, 1]]*10
y = [0, 1, 1, 0]*10

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X, y)

[clf.predict([sample]) for sample in [(0, 0), (0, 1), (1, 0), (1, 1)]]

[array([0]), array([0]), array([0]), array([0])]

This is wrong - presumably because a linear model simply isn't powerful enough, whereas something like RandomForest is:

In [22]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X, y)

[clf.predict([sample]) for sample in [(0, 0), (0, 1), (1, 0), (1, 1)]]

[array([0]), array([1]), array([1]), array([0])]

So RandomForest clearly works, but linear models do not. So we need to add some power to our model. How can we change it to be more powerful? Some ideas that come to mind:

* Move from a linear to a polynomial model.
* Add another layer to the network.

Would these work? Is it possible that adding layers is similar in function to adding polynomials?

Let's go with adding a layer...

So the very simplest layer I could think of would be this:

<img src="images/xor_network2.png"  style="width: 50%; height: 50%;" />
(note: weights should be w0, w1, w2, and w3)

But I know that typically, when I've seen pictures of neural networks, the inputs from one layer go to each of the nodes in the next layer - more like this:

<img src="images/xor_network3.png"  style="width: 50%; height: 50%;" />
(note: weights should be w0, w1, w2, and w3)

I think the reason for this is that, in the prior network (without the cross in the middle), the 2 layers of weights are still only being added to the same input, which is really functionally the same as just having 1 layer. So I'm going to test out the network with the cross.

So now we have 4 weights to play with. Let's see if I could imagine a solution where the weights are adjusted so that xor works... on second thought, that sounds like a good job for a program - let's do it that way.

### Finding weights programmatically

A few initial thoughts on writing a program to learn:
* The inputs for the second layer are not the inputs, but the outputs of the previous layer.
* We probably can't just keep 4 variables (one for each weight we are trying to tune), because we might optimize for the first sample, and then reoptimize for the second, and third, and so forth. One solution would be to generate a number of hypotheses for the weights, and keep them all in memory, and then find the one that minimizes error for all samples. Later, we can try to actually tweak the weights.

So let's try to write this. First, some helper functions...

In [161]:
import random

def rand_weight(weight_min, weight_max):
    """Generates a uniformly random weight between weight_min and weight_max."""
    return random.random() * (weight_max - weight_min) + weight_min

def gen_hypothesis(weight_min, weight_max, num_weights):
    return tuple([rand_weight(weight_min, weight_max) for i in range(num_weights)])
        

In [162]:
rand_weight(0, 10)

5.938529171631436

In [166]:
gen_hypothesis(-5, 5, 4)

(-3.0073873187744695,
 -1.0547383843095992,
 4.059153250556346,
 -4.575705844790714)

### Node functionality

Next, let's define the functionality of each node. I know there's the concept of an [Activation function](https://en.wikipedia.org/wiki/Activation_function) in common neural network terminology. I suppose there are several ways a given node could function:
* It could always output some discrete value (like 1 and 0, or -1 and 1).
* It could output a continuous value bounded within some range like a probability (e.g. 0 to 1).
* It could output an unbounded continuous value.

For starters, I'm going to just go with what seems the simplest - it's just going to output an unbounded continuous value taking by multiplying the input by the weight. So the equation to determine the output of a given node will be: $node(x) = w0 * x$.

On second thought, though... now that we have a hidden layer, those nodes will have more than one input. So how should that be handled? It could be handled a few ways:
* We could make each input have its own weight.
* We could use the same weight for all of the inputs.

I'm again going to choose the dumbest easiest approach and say each node only has a single weight (even though, if I recall correctly, neural networks typically have different weights for each input - but we'll get there eventually). And if there is more than 1 sample, we'll just add them all together.

In [180]:
class Node:
    def __init__(self, weight):
        self.weight = weight
    def run(self, node_input):
        if type(node_input) != list:
            node_input = [node_input]
        return sum([self.weight * input_val for input_val in node_input])

Let's test the node implementation...

In [181]:
n = Node(10)
[n.run([1]), n.run([5]), n.run([1, 2, 3])]

[10, 50, 60]

Looks good. Now let's try to write the network.

Actually, just trying to write this, I immediately realize that we need to do some sort of convolution at the output node (to join its 2 inputs), so let's just add a 5th weight for that node.

In [182]:
def neural_net_1(weights, sample):
    input_layer = [Node(weights[0]), Node(weights[1])]
    hidden_layer = [Node(weights[3]), Node(weights[2])]
    output_layer = Node(weights[4])
    
    input_layer_output = [input_layer[i].run(sample[i]) for i in range(len(input_layer))]
    hidden_layer_output = [hidden_layer[i].run(input_layer_output) for i in range(len(hidden_layer))]
    output_layer_output = output_layer.run(hidden_layer_output)
    
    return output_layer_output

Let's test that...

In [184]:
neural_net_1([0, 0, 0, 0, 0], [1, 1, 1, 1, 1])

0

In [185]:
neural_net_1([1, 1, 1, 1, 1], [1, 1, 1, 1, 1])

4

In [187]:
# Should double the above
neural_net_1([1, 1, 1, 1, 2], [1, 1, 1, 1, 1])

8

In [186]:
neural_net_1([2, 2, 2, 2, 2], [10, 10, 10, 10, 10])

320

Seems to work as expected. Let's now write an algorithm to generate a bunch of weight hypotheses, and choose the best one.

In [214]:
class NeuralNetPredictor:
    def __init__(self, num_hypotheses=1000, weight_min=-5, weight_max=5):
        # generate hypotheses randomly
        self.hypotheses = [gen_hypothesis(weight_min, weight_max, 5) for i in range(num_hypotheses)]
        self.hypothesis_to_error = {h: 0 for h in self.hypotheses}
        self.trained_weights = [0]*5
    
    def add_sample(self, sample, expected_value):
        for h in self.hypotheses:
            network_value = neural_net_1(h, sample)
            # print('sample = {}, network_value = {}'.format(sample, network_value))
            err = (network_value - expected_value)**2
            self.hypothesis_to_error[h] += err
    
    def train(self, samples, labels):
        for index in range(len(samples)):
            self.add_sample(samples[index], labels[index])
        
        # Find hypothesis with lowest error
        best_hypothesis = self.hypotheses[0]
        lowest_err = 10000000
        for hyp_index in range(len(self.hypotheses)):
            hyp = self.hypotheses[hyp_index]
            err = self.hypothesis_to_error[hyp]
            if err < lowest_err:
                lowest_err = err
                best_hypothesis = hyp
        
        self.trained_weights = best_hypothesis
        print('Best hypothesis weights: {}'.format(best_hypothesis))
 
    def predict(self, sample):
        return neural_net_1(self.trained_weights, sample)

In [220]:
# Note that I'm making 10x repetitions of the logic table.
X = [[0, 0],
     [0, 1],
     [1, 0],
     [1, 1]]*10
y = [0, 1, 1, 0]*10

xor_predictor = NeuralNetPredictor()
xor_predictor.train(X, y)
print('Predictions: {}'.format([xor_predictor.predict(x) for x in [(0, 0), (0, 1), (1, 0), (1, 1)]]))

Best hypothesis weights: (-3.448188486114756, -2.906375898177732, 1.2268473056308373, -1.2651189785360972, 2.685541103398994)
Predictions: [0.0, 0.2987177527559055, 0.3544053321859728, 0.6531230849418748]


In [221]:
xor_predictor = NeuralNetPredictor()
xor_predictor.train(X, y)
print('Predictions: {}'.format([xor_predictor.predict(x) for x in [(0, 0), (0, 1), (1, 0), (1, 1)]]))

Best hypothesis weights: (-3.203594529830033, -4.496694842845448, 2.4246931548759516, -2.4409435203483087, 4.405656218287904)
Predictions: [0.0, 0.32193422877027444, 0.2293566208733253, 0.5512908496435927]


In [222]:
xor_predictor = NeuralNetPredictor()
xor_predictor.train(X, y)
print('Predictions: {}'.format([xor_predictor.predict(x) for x in [(0, 0), (0, 1), (1, 0), (1, 1)]]))

Best hypothesis weights: (-4.93163851348113, -4.501216296913628, -4.806145566496756, 4.912783016586257, -0.5993876984644997)
Predictions: [0.0, 0.2877050332703366, 0.31521640574598564, 0.602921439016324]


### It doesn't work!

Okay, so this is clearly not working. At this point, we're not using converting into classes based on the output value, but still, I'd expect the output values to be higher at (0, 1) and (1, 0). But instead, it seems like they pretty much correspond to the input values (1, 1) is almost always the highest, (0, 0) is always 0, and (0, 1) and (1, 0) are almost always about half of (1, 1). Interestingly, the 4 classes summed together is very near 1 - which is kind of surprising considering I'm not explicitly doing any probability normalizations.

So why is this not working? My guess is that, since the "expected value" used to calculate the error with is always either 0 or 1, it's being almost entirely factored out by large network output values. So to fix this, we'd probably want the network values to be something in the 0 to 1 range (to match the classes).

Well, we'll try that another day.